In [1]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [3]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_fe_v05.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_fe_v05.parquet...
- [1 files][  9.5 GiB/  9.5 GiB]                                                
Operation completed over 1 objects/9.5 GiB.                                      


In [4]:
base_path = '/home/eanegrin/'
dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_fe_v05.parquet'

semillas = [122219, 109279, 400391, 401537, 999961]

In [5]:
data = pd.read_parquet(dataset_path + dataset_file)

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [7]:
meses_excluidos = [201901,201902,201903,201904,201905,
                   202006,
                   202108, 202109] # meses con clase ternaria incompleta

data = data[~data['foto_mes'].isin(meses_excluidos)]
data['foto_mes'].unique()

array([201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001,
       202002, 202003, 202004, 202005, 202007, 202008, 202009, 202010,
       202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106,
       202107])

In [8]:
data.shape

(3784752, 1278)

In [9]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.995294,0.004706
1,201907,0.994729,0.005271
2,201908,0.995839,0.004161
3,201909,0.995712,0.004288
4,201910,0.995435,0.004565
5,201911,0.994700,0.005300
6,201912,0.995749,0.004251
7,202001,0.996513,0.003487
8,202002,0.998742,0.001258
9,202003,0.997469,0.002531


In [10]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    # print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 20

    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rearmar
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes

    df_subsampled.append(group_resampled)

# Mergear
data = pd.concat(df_subsampled, ignore_index=True)

data.shape

(206434, 1278)

In [11]:
# proporciones luego de hacer el ajuste

proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.913966,0.086034
1,201907,0.904637,0.095363
2,201908,0.923173,0.076827
3,201909,0.920998,0.079002
4,201910,0.916331,0.083669
5,201911,0.904147,0.095853
6,201912,0.921635,0.078365
7,202001,0.934805,0.065195
8,202002,0.975464,0.024536
9,202003,0.951816,0.048184


In [12]:
output_file = 'competencia_03_fe_v05_undersampled.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [13]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v05_undersampled.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v05_undersampled.parquet...
- [1 files][567.9 MiB/567.9 MiB]                                                
Operation completed over 1 objects/567.9 MiB.                                    
